In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

db_path = 'postgres:///music'
engine = create_engine(db_path)

table_names = engine.table_names()

table_names

In [5]:
results = []

In [6]:
results = [conn.execute(f'''
    SELECT * FROM {x}''') for x in table_names]

In [37]:

def postgres_to_df(table):
    engine = create_engine(db_path)
    conn = engine.connect()
    result = conn.execute(f'''
    SELECT * FROM {table}''')
    columns = [x for x in result.keys()]
    df = pd.DataFrame(result.fetchall(),columns=columns)
    return df

In [64]:
postgres_to_df('reviews')

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017
5,22722,insecure (music from the hbo original series),various artists,http://pitchfork.com/reviews/albums/22722-inse...,7.4,0,vanessa okoth-obbo,contributor,2017-01-05,3,5,1,2017
6,22704,stillness in wonderland,little simz,http://pitchfork.com/reviews/albums/22704-litt...,7.1,0,katherine st. asaph,contributor,2017-01-05,3,5,1,2017
7,22694,tehillim,yotam avni,http://pitchfork.com/reviews/albums/22694-tehi...,7.0,0,andy beta,contributor,2017-01-05,3,5,1,2017
8,22714,reflection,brian eno,http://pitchfork.com/reviews/albums/22714-refl...,7.7,0,andy beta,contributor,2017-01-04,2,4,1,2017
9,22724,filthy america its beautiful,the lox,http://pitchfork.com/reviews/albums/22724-filt...,5.3,0,ian cohen,contributor,2017-01-04,2,4,1,2017


# Is there a statistical difference between the ratings of two different music genres: Jazz and Metal? Two tailed test
H<sub>0</sub>: Songs of the Jazz Genre will not have a different rating allocation from Metal
# Will the label with the most artists have the highests scores? One tailed test
H<sub>0</sub>: Will the label with the most artists have higher scores than the label with the least artists signed?
# Is there a relationship between labels scores and genres? One tailed test
H<sub>0</sub>: The label with the most genre diversity will not have higher scores than the label with least genre diversity
# Have average song ratings changed from 2000-2009 to 2010-2017?
H<sub>0</sub>: Average song ratings have stayed the same year over year
